In [1]:
import pandas as pd
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

# check if GLPK is installed. If not, install.
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk
        except:
            pass
assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

# Formulación de mezclas óptima
Dado un conjunto de productos químicos y un conjunto de restricciones de propiedades, determinar la mezcla y/o combinación óptima de los diferentes compuestos que resulta en el menor coste posible.

## Datos

In [2]:
d = {'Coste':{'A': 2. , 'B': 2.  , 'C': 5.  }, 
     'VitA': {'A': .5 ,'B': .4, 'C': .3 }, 
     'VitB': {'A': .2 , 'B': .1 , 'C': .3 }
     }
df = pd.DataFrame.from_dict(d, orient='index')

display(df)
print('Cantidad máxima de vitamina A:', 0.4)
print('Cantidad mínima de vitamina B:', 0.2)
print('Componenetes A y B son incompatibles')

,A,B,C
Coste,2.0,2.0,5.0
VitA,0.5,0.4,0.3
VitB,0.2,0.1,0.3


Cantidad máxima de vitamina A: 0.4
Cantidad mínima de vitamina B: 0.2
Componenetes A y B son incompatibles


In [3]:
data = {None: {'C': ['A','B','C'] , 
               'P': d['Coste'] , 
               'VitA': d['VitA'] ,
               'VitB': d['VitB']  ,
               'VitA_UB': .4 ,
               'VitB_LB': .2 , 
               }} 

## Implementación del modelo
El primer paso consiste en importar Pyomo y definir el tipo de modelo que se va a utilizar. Para este problema vamos a utilizar un modelo tipo `ConcreteModel`.

In [4]:
import pyomo.environ as pyo

model  = pyo.ConcreteModel()

### Sets

In [5]:
model.C = pyo.Set(initialize = ['A','B','C'], doc = 'Componentes') 
def incompatible_filter(model, i, j):
    return i == 'A' and j == 'B' 
model.C_incompat = pyo.Set(initialize=model.C*model.C, filter=incompatible_filter, doc = 'Pares incompatibles')

In [6]:
model.C_incompat.pprint()
model.C.pprint()

C_incompat : Pares incompatibles
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :    1 : {('A', 'B'),}
C : Componentes
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    3 : {'A', 'B', 'C'}


### Variables

In [7]:
model.x = pyo.Var(model.C, bounds = (0.,1.))
model.y = pyo.Var(model.C_incompat, domain=pyo.Boolean)

### Parámetros

In [8]:
model.P = pyo.Param(model.C, initialize = d['Coste'], doc='Coste UM')
model.VitA = pyo.Param(model.C, initialize = d['VitA'], doc='Contenido de VitA')
model.VitB = pyo.Param(model.C, initialize = d['VitB'], doc='Contenido de VitB')
model.VitA_UB = pyo.Param(initialize = 0.4, doc='Cantidad máxima de VitA')
model.VitB_LB = pyo.Param(initialize = 0.2, doc='Cantidad mínima de VitB')
model.BigM = pyo.Param(initialize = 100, doc = 'Coeficiente BigM')

### Funcion objetivo
$$
\begin{align}
\text{Coste total} & = \sum_{c\in C} x_{c} P_{c} \nonumber
\end{align}
$$

In [9]:
def obj_rule(m):
    return sum(m.P[c]*m.x[c] for c in m.C)
model.obj = pyo.Objective(rule = obj_rule)

### Restricciones estructurales
- Balance de materia
$
\begin{align}
\sum_{c\in C} x_{c} & = 1 \nonumber
\end{align}
$

In [10]:
model.massfraction = pyo.Constraint(rule = lambda m: sum(m.x[c] for c in m.C) == 1)

- Especificaciones del producto final
$
\begin{align}
\sum_{c\in C} x_{c}VitA_{c} & \leq VitA^{UB} \nonumber \\
\sum_{c\in C} x_{c}VitB_{c} & \geq VitB^{LB} \nonumber
\end{align}
$

In [11]:
model.comp_ub = pyo.Constraint(rule = lambda m: sum(m.VitA[c]*m.x[c] for c in m.C) <= m.VitA_UB )
model.comp_lb = pyo.Constraint(rule = lambda m: sum(m.VitB[c]*m.x[c] for c in m.C) >= m.VitB_LB ) 

# Restricciones lógicas: incompatibilidad de componentes

- Reformulación Big-M
$
\begin{align}
x_A & \leq M\;y \nonumber \\
x_B & \leq (1-M)\;y \nonumber
\end{align}
$


In [12]:
model.ref_BigM = pyo.ConstraintList()
for pair in model.C_incompat:
    a, b = pair
    model.ref_BigM.add(model.x[a] <= model.BigM*model.y[pair])
    model.ref_BigM.add(model.x[b] <= model.BigM*(1-model.y[pair]))        

- Disyunciones

Antes de utilizar la enxtensión para GDP de Pyomo, hay que importar las funciones correspondientes. Para modelar la siguiente disyunción, se necesitarán los comandos `Disjunct`, `Disjunction` y, más adelante, `TransformationFactory`.

$$
\left[
\begin{array}{c}
Y \\
x_B=0
\end{array}
\right] 
\lor 
\left[
\begin{array}{c}
\neg Y \\
x_A=0
\end{array}
\right]
$$

In [13]:
# Importar funciones necesarias
from pyomo.gdp import Disjunct, Disjunction

# Término que se cumple si Y = True
model.term1 = Disjunct()
model.term1.consA = pyo.Constraint(expr= model.x['A'] == 0)

# Término que se cumple si Y = False
model.term2 = Disjunct()
model.term2.consB = pyo.Constraint(expr= model.x['B'] == 0)

# Añadir restricción en forma de disyunción
model.disyuncion = Disjunction(expr = [model.term1, model.term2])

## Resolución
El solver que se va a utilizar en la optimización de este problema es Cbc (Coin-or branch and cut). La nomenclatura para llamarlo es `SolverFactory('cbc')`.

### Reformulación Big M

In [14]:
# Desactivar restricciones
model.term1.deactivate()
model.term2.deactivate()
model.disyuncion.deactivate()

# Llamada al solver
opt = pyo.SolverFactory('glpk')
results = opt.solve(model)

In [15]:
for c in model.C:
    print(f"{c} = {model.x[c]()}")

A = 0.5
B = 0.0
C = 0.5


### Extensión Pyomo GDP
Aplica el método de la envolvente convexa para reformular la disyunción que aparece en este problema. La sintaxis que debes utilizar es `TransoformationFactory(trf_name).apply_to(model_name)`.

In [16]:
# # Acivar/desactivar restricciones
# model.term1.activate()
# model.term2.activate()
model.disyuncion.activate()

model.ref_BigM.deactivate()

# Aplicar tranformación
transformation = 'hull'
if transformation == 'bigm':
    pyo.TransformationFactory('gdp.bigm').apply_to(model)
elif transformation == 'hull':
    pyo.TransformationFactory('gdp.hull').apply_to(model)
  
# Llamada al solver
opt = pyo.SolverFactory('glpk')
results = opt.solve(model)

In [17]:
for c in model.C:
    print(f"{c} = {model.x[c]()}")

A = 0.5
B = 0.0
C = 0.5
